# Лабораторная работа №2

**Требования:**
* Python >= 3.X

Лабораторную работу необходимо выполнять в данном шаблоне. Результатом работы будет являться файл (с измененным именем), который необходимо выложить в Moodle.

**Важно!!!** Имя файлу задавайте по следующему шаблону **lab_2_6231-010402D_Кришталь К.Е.ipynb**. Например: если Вас зовут Иванов Иван Иванович, и Вы обучаетесь в группе 6207_010302D, то имя файла будет выглядеть так **lab_2_6231-010402D_Кришталь К.Е..ipynb**.

Необходимо провести исследование различных способов представления документов и их влияние на качество определения тональности.

В качестве входных данных к лабораторной работе взят широко известный набор данных IMDB, содержащий 50K обзоров фильмов ([imdb-dataset-of-50k-movie-reviews](https://disk.yandex.ru/i/DDb0zuyUmts5QA)). Откликами являются значения двух классов positive и negative.

In [17]:
#!pip install --upgrade nltk


In [19]:
#!pip install tensorflow


In [3]:
# Код загрузки данных
# Если хотите добавить какие-либо библиотеки
# добавляйте их ИМЕННО ЗДЕСЬ
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
imdb_data = pd.read_csv(r'input/IMDB Dataset.csv')
imdb_data.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


#### Шаг №1 Подготовка данных

Обязательно предобработайте данные!



In [5]:
# Проверка на пустые строки и удаление
imdb_data = imdb_data.dropna(subset=['review'])

# Функция для очистки текста
import re

def clean_text(text):
    # Удаляем цифры, знаки препинания, приводим к нижнему регистру
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # оставляем только буквы
    text = text.lower()  # делаем маленькие буквы
    return text

# Очищаем все отзывы
imdb_data['cleaned_review'] = imdb_data['review'].apply(clean_text)


print("Очищенные отзывы:")
print(imdb_data['cleaned_review'].head())

# Удаление стоп-слов
def remove_stopwords(text):
    words = text.split()  # разбиваем текст на слова
    words_filtered = [word for word in words if word not in stop_words]  # удаляем стоп-слова
    return ' '.join(words_filtered)  # обратно в строку

# Удаляем стоп-слова
imdb_data['review_without_stopwords'] = imdb_data['cleaned_review'].apply(remove_stopwords)

# Вывод отзывов после удаления стоп-слов
print("Отзывы после удаления стоп-слов:")
print(imdb_data['review_without_stopwords'].head())

# Лемматизация (если нужно, можно раскомментировать код)
# lemmatizer = WordNetLemmatizer()

# def lemmatize_text(text):
#     words = text.split()  # разбиваем текст на слова
#     lemmatized_words = [lemmatizer.lemmatize(word) for word in words]  # лемматизируем каждое слово
#     return ' '.join(lemmatized_words)  # соединяем обратно в строку

# imdb_data['lemmatized_review'] = imdb_data['review_without_stopwords'].apply(lemmatize_text)

# Вывод лемматизированных отзывов
# print("Лемматизированные отзывы:")
# print(imdb_data['lemmatized_review'].head())

# imdb_data.head(223)


Очищенные отзывы:
0    one of the other reviewers has mentioned that ...
1    a wonderful little production br br the filmin...
2    i thought this was a wonderful way to spend ti...
3    basically theres a family where a little boy j...
4    petter matteis love in the time of money is a ...
Name: cleaned_review, dtype: object
Отзывы после удаления стоп-слов:
0    one reviewers mentioned watching oz episode yo...
1    wonderful little production br br filming tech...
2    thought wonderful way spend time hot summer we...
3    basically theres family little boy jake thinks...
4    petter matteis love time money visually stunni...
Name: review_without_stopwords, dtype: object


При векторизации ограничьте количество признаков до 500. В качестве исследуемых способов векторизации текстов необходимо рассмотреть:

#### 1. Компоненты вектора: частоты ([CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)).

In [8]:
# вставьте код здесь


# Создаем CountVectorizer с ограничением до 500 признаков
vectorizer_count = CountVectorizer(max_features=500)

# Применяем векторизацию к текстам
X_count = vectorizer_count.fit_transform(imdb_data['review_without_stopwords'])

# Преобразуем результат в DataFrame для удобства
X_count_df = pd.DataFrame(X_count.toarray(), columns=vectorizer_count.get_feature_names_out())

# Вывод первых 5 строк DataFrame
print(X_count_df.head())

   able  absolutely  across  act  acting  action  actor  actors  actress  \
0     0           0       0    0       0       0      0       0        0   
1     0           0       0    0       0       0      0       1        0   
2     0           0       0    0       0       0      0       0        0   
3     0           0       0    0       0       0      0       0        0   
4     0           0       0    0       1       1      0       0        0   

   actually  ...  writing  written  wrong  year  years  yes  yet  youll  \
0         0  ...        0        0      0     0      0    0    0      1   
1         0  ...        0        1      0     0      0    0    0      0   
2         0  ...        0        0      0     0      1    0    0      0   
3         0  ...        0        0      0     0      0    0    0      0   
4         0  ...        0        0      0     0      0    0    0      0   

   young  youre  
0      0      0  
1      0      0  
2      1      0  
3      0      1  
4 

#### 2. Компоненты вектора: оценки tf-idf для слова ([TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)). 

In [20]:

# Применяем TfidfVectorizer к текстам без стоп-слов
X_tfidf = tfidf_vectorizer.fit_transform(imdb_data['review_without_stopwords'])

# Преобразуем результат в DataFrame
X_tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())


print(X_tfidf_df.head())


   able  absolutely  across  act    acting    action  actor    actors  \
0   0.0         0.0     0.0  0.0  0.000000  0.000000    0.0  0.000000   
1   0.0         0.0     0.0  0.0  0.000000  0.000000    0.0  0.112843   
2   0.0         0.0     0.0  0.0  0.000000  0.000000    0.0  0.000000   
3   0.0         0.0     0.0  0.0  0.000000  0.000000    0.0  0.000000   
4   0.0         0.0     0.0  0.0  0.072111  0.097863    0.0  0.000000   

   actress  actually  ...  writing   written  wrong  year    years  yes  yet  \
0      0.0       0.0  ...      0.0  0.000000    0.0   0.0  0.00000  0.0  0.0   
1      0.0       0.0  ...      0.0  0.151696    0.0   0.0  0.00000  0.0  0.0   
2      0.0       0.0  ...      0.0  0.000000    0.0   0.0  0.14788  0.0  0.0   
3      0.0       0.0  ...      0.0  0.000000    0.0   0.0  0.00000  0.0  0.0   
4      0.0       0.0  ...      0.0  0.000000    0.0   0.0  0.00000  0.0  0.0   

      youll     young     youre  
0  0.114195  0.000000  0.000000  
1  0.000000 

#### 3. Компоненты вектора: частоты N-грам.

In [24]:
# вставьте код здесь

# Векторизация текста с униграммами и биграммами
vectorizer_ngram = CountVectorizer(ngram_range=(2, 2), max_features=500)

# Используем 'review_without_stopwords' как очищенные данные
X_ngram = vectorizer_ngram.fit_transform(imdb_data['review_without_stopwords'])

# Преобразуем результат в DataFrame
X_ngram_df = pd.DataFrame(X_ngram.toarray(), columns=vectorizer_ngram.get_feature_names_out())

# Вывод первых 5 строк
print(X_ngram_df.head(5))


   absolutely nothing  acting bad  acting good  action movie  action scenes  \
0                   0           0            0             0              0   
1                   0           0            0             0              0   
2                   0           0            0             0              0   
3                   0           0            0             0              0   
4                   0           0            1             0              0   

   action sequences  againbr br  allbr br  almost every  along way  ...  \
0                 0           0         0             0          0  ...   
1                 0           0         0             0          0  ...   
2                 0           0         0             0          0  ...   
3                 0           0         0             0          0  ...   
4                 0           0         0             0          0  ...   

   years ago  years later  years old  yet another  young girl  young man  

### Шаг 2. Исследование моделей

Матрица ошибок (confusion matrix):

<table>
		<tr>
			<td></td>
			<td>$y = 1$</td>
			<td>$y = 0$</td>
		</tr>
		<tr>
			<td>$a(x) = 1$</td>
			<td>True Positive (TP)</td>
			<td>False Positive (FP)</td>
		</tr>
    	<tr>
			<td>$a(x) = 0$</td>
			<td>False Negative (FN)</td>
			<td>True Negative (TN)</td>
		</tr>
</table>

Метрики качества классификации:

$$\operatorname{accuracy} = \frac{\operatorname{TP} + \operatorname{TN}}{\operatorname{TP} + \operatorname{TN} + \operatorname{FP} + \operatorname{FN}}$$


$$\operatorname{precision} = \frac{\operatorname{TP}}{\operatorname{TP} + \operatorname{FP}}$$

$$\operatorname{recall} = \frac{\operatorname{TP}}{\operatorname{TP} + \operatorname{FN}}$$

$$\operatorname{F} = \frac{\operatorname{precision} \cdot \operatorname{recall}}{\operatorname{precision} + \operatorname{recall}}$$

Для каждой модели и каждого способа векторизации текстов необходимо:

1. Определить оптимальные гиперпараметры (по F-мере) ([GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)).

2. Количество блоков при перекрестной проверке (cross-validation) должно быть равно 3.

3. Для ускорения процесса можно ограничиться 30% всех данных.

Оценку производим для следующих моделей:

#### 1. Машина опорных векторов ([SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)).

In [26]:
# вставьте код здесь
param_grid_svc = {
    'C': [0.2, 2, 20],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}


In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report, confusion_matrix

# # Векторизация с использованием TF-IDF
# tfidf_vectorizer = TfidfVectorizer(max_features=500)
# X_tfidf = tfidf_vectorizer.fit_transform(X)

# # Векторизация с использованием CountVectorizer
# count_vectorizer = CountVectorizer(max_features=500)
# X_count = count_vectorizer.fit_transform(X)

# # Разделение данных на обучающие и тестовые
# X_train_tfidf, X_test_tfidf, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)
# X_train_count, X_test_count, y_train_count, y_test_count = train_test_split(X_count, y, test_size=0.2, random_state=42)

# # Преобразование в тензоры для использования с PyTorch и CUDA
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# X_train_tfidf_tensor = torch.tensor(X_train_tfidf.toarray(), dtype=torch.float32).to(device)
# X_test_tfidf_tensor = torch.tensor(X_test_tfidf.toarray(), dtype=torch.float32).to(device)
# y_train_tensor = torch.tensor(y_train.values).long().to(device)
# y_test_tensor = torch.tensor(y_test.values).long().to(device)

# X_train_count_tensor = torch.tensor(X_train_count.toarray(), dtype=torch.float32).to(device)
# X_test_count_tensor = torch.tensor(X_test_count.toarray(), dtype=torch.float32).to(device)
# y_train_count_tensor = torch.tensor(y_train_count.values).long().to(device)
# y_test_count_tensor = torch.tensor(y_test_count.values).long().to(device)


In [ ]:
# class TextClassifier(nn.Module):
#     def __init__(self, input_size):
#         super(TextClassifier, self).__init__()
#         self.linear = nn.Linear(input_size, 2)  # 2 выхода для бинарной классификации
    
#     def forward(self, x):
#         return self.linear(x)

# # Модели для обеих векторизаций
# input_size_tfidf = X_train_tfidf_tensor.shape[1]
# input_size_count = X_train_count_tensor.shape[1]

# model_tfidf = TextClassifier(input_size_tfidf).to(device)
# model_count = TextClassifier(input_size_count).to(device)

# # Функция потерь и оптимизатор
# criterion = nn.CrossEntropyLoss()
# optimizer_tfidf = optim.Adam(model_tfidf.parameters(), lr=0.001)
# optimizer_count = optim.Adam(model_count.parameters(), lr=0.001)


In [ ]:
# def train_model(model, optimizer, X_train, y_train, num_epochs=10):
#     model.train()
#     for epoch in range(num_epochs):
#         optimizer.zero_grad()
#         outputs = model(X_train)
#         loss = criterion(outputs, y_train)
#         loss.backward()
#         optimizer.step()
#         print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# # Обучение модели с TF-IDF
# train_model(model_tfidf, optimizer_tfidf, X_train_tfidf_tensor, y_train_tensor)

# # Обучение модели с CountVectorizer
# train_model(model_count, optimizer_count, X_train_count_tensor, y_train_count_tensor)


In [ ]:
# def evaluate_model(model, X_test, y_test):
#     model.eval()
#     with torch.no_grad():
#         outputs = model(X_test)
#         _, predicted = torch.max(outputs.data, 1)
    
#     predicted = predicted.cpu()
#     y_test_cpu = y_test.cpu()

#     print("Confusion Matrix:")
#     print(confusion_matrix(y_test_cpu, predicted))
#     print("Classification Report:")
#     print(classification_report(y_test_cpu, predicted))

# print("Оценка модели с TF-IDF:")
# evaluate_model(model_tfidf, X_test_tfidf_tensor, y_test_tensor)

# print("Оценка модели с CountVectorizer:")
# evaluate_model(model_count, X_test_count_tensor, y_test_count_tensor)


#### 2. Случайный лес ([RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)).

In [ ]:

# Разделение данных (30% тест, 70% обучающие данные)
X = imdb_data['review_without_stopwords']  
y = imdb_data['sentiment']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:
# ----------- TF-IDF Векторизация -----------
print("\nРезультаты для TF-IDF векторизации:")

tfidf_vectorizer = TfidfVectorizer(max_features=500)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# ----------- GridSearch для SVC -----------
# Параметры для GridSearchCV (для SVC)
param_grid_svc = {
    'C': [0.2, 2, 20],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

# Обучение SVC модели с использованием GridSearch
svc_model = SVC()
grid_search_svc = GridSearchCV(svc_model, param_grid_svc, scoring='f1_macro', cv=2, n_jobs=-1)
grid_search_svc.fit(X_train_tfidf, y_train)

# Лучшая модель SVC
svc_best = grid_search_svc.best_estimator_
y_pred_svc = svc_best.predict(X_test_tfidf)

# Метрики для SVC
print("\nКлассификация SVC с TF-IDF:")
print(classification_report(y_test, y_pred_svc))
print(confusion_matrix(y_test, y_pred_svc))
print("Лучшие параметры для SVC:", grid_search_svc.best_params_)
print("Лучший результат (F1):", grid_search_svc.best_score_)

In [ ]:

# ----------- CountVectorizer Векторизация -----------
print("\nРезультаты для CountVectorizer векторизации:")

count_vectorizer = CountVectorizer(max_features=500)
X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

# Создаем модель случайного леса с CountVectorizer
rf_model_count = RandomForestClassifier(random_state=42)

# Выполнение GridSearchCV для CountVectorizer
grid_search_rf_count = GridSearchCV(rf_model_count, param_grid_rf, scoring='f1_macro', cv=2, n_jobs=-1)
grid_search_rf_count.fit(X_train_count, y_train)

# Лучшая модель для CountVectorizer
rf_best_count = grid_search_rf_count.best_estimator_
y_pred_rf_count = rf_best_count.predict(X_test_count)

# Метрики для CountVectorizer
print("\nКлассификация RandomForest с CountVectorizer:")
print(classification_report(y_test, y_pred_rf_count))
print(confusion_matrix(y_test, y_pred_rf_count))
print("Лучшие параметры для CountVectorizer:", grid_search_rf_count.best_params_)
print("Лучший результат (F1):", grid_search_rf_count.best_score_)


In [31]:
#вЗЯТЬ 30% ОТ ПРЕДЕДЫЩИХ ДАННЫХ

from sklearn.svm import SVC

X = imdb_data['review_without_stopwords']  
y = imdb_data['sentiment']  

# 30 на 70
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# ----------- TF-IDF Векторизация -----------
print("\nРезультаты для TF-IDF векторизации:")

tfidf_vectorizer = TfidfVectorizer(max_features=500)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# ----------- GridSearch для SVC -----------
# Параметры для GridSearchCV (для SVC)
param_grid_svc = {
    'C': [0.2, 2, 20],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

# Обучение SVC модели с использованием GridSearch
svc_model = SVC()
grid_search_svc = GridSearchCV(svc_model, param_grid_svc, scoring='f1_macro', cv=2, n_jobs=-1)
grid_search_svc.fit(X_train_tfidf, y_train)

# Лучшая модель SVC
svc_best = grid_search_svc.best_estimator_
y_pred_svc = svc_best.predict(X_test_tfidf)

# Метрики для SVC
print("\nКлассификация SVC с TF-IDF:")
print(classification_report(y_test, y_pred_svc))
print(confusion_matrix(y_test, y_pred_svc))
print("Лучшие параметры для SVC:", grid_search_svc.best_params_)
print("Лучший результат (F1):", grid_search_svc.best_score_)

# ----------- RandomForest для TF-IDF -----------
print("\nРезультаты для RandomForest с TF-IDF:")

# Определение параметров для GridSearchCV для RandomForest
param_grid_rf = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]
}

# Обучение модели RandomForest с использованием GridSearch
rf_model_tfidf = RandomForestClassifier(random_state=42)
grid_search_rf_tfidf = GridSearchCV(rf_model_tfidf, param_grid_rf, scoring='f1_macro', cv=2, n_jobs=-1)
grid_search_rf_tfidf.fit(X_train_tfidf, y_train)

# Лучшая модель для TF-IDF
rf_best_tfidf = grid_search_rf_tfidf.best_estimator_
y_pred_rf_tfidf = rf_best_tfidf.predict(X_test_tfidf)

# Метрики для TF-IDF
print("\nКлассификация RandomForest с TF-IDF:")
print(classification_report(y_test, y_pred_rf_tfidf))
print(confusion_matrix(y_test, y_pred_rf_tfidf))
print("Лучшие параметры для RandomForest:", grid_search_rf_tfidf.best_params_)
print("Лучший результат (F1):", grid_search_rf_tfidf.best_score_)

# ----------- CountVectorizer Векторизация -----------
print("\nРезультаты для CountVectorizer векторизации:")

count_vectorizer = CountVectorizer(max_features=500)
X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

# Создаем модель случайного леса с CountVectorizer
rf_model_count = RandomForestClassifier(random_state=42)

# Выполнение GridSearchCV для CountVectorizer
grid_search_rf_count = GridSearchCV(rf_model_count, param_grid_rf, scoring='f1_macro', cv=2, n_jobs=-1)
grid_search_rf_count.fit(X_train_count, y_train)

# Лучшая модель для CountVectorizer
rf_best_count = grid_search_rf_count.best_estimator_
y_pred_rf_count = rf_best_count.predict(X_test_count)

# Метрики для CountVectorizer
print("\nКлассификация RandomForest с CountVectorizer:")
print(classification_report(y_test, y_pred_rf_count))
print(confusion_matrix(y_test, y_pred_rf_count))
print("Лучшие параметры для CountVectorizer:", grid_search_rf_count.best_params_)
print("Лучший результат (F1):", grid_search_rf_count.best_score_)



Результаты для TF-IDF векторизации:

Классификация SVC с TF-IDF:
              precision    recall  f1-score   support

    negative       0.85      0.83      0.84      7411
    positive       0.84      0.86      0.85      7589

    accuracy                           0.85     15000
   macro avg       0.85      0.85      0.85     15000
weighted avg       0.85      0.85      0.85     15000

[[6183 1228]
 [1083 6506]]
Лучшие параметры для SVC: {'C': 2, 'gamma': 'scale', 'kernel': 'rbf'}
Лучший результат (F1): 0.8341336659296981

Результаты для RandomForest с TF-IDF:

Классификация RandomForest с TF-IDF:
              precision    recall  f1-score   support

    negative       0.82      0.81      0.81      7411
    positive       0.81      0.83      0.82      7589

    accuracy                           0.82     15000
   macro avg       0.82      0.82      0.82     15000
weighted avg       0.82      0.82      0.82     15000

[[5969 1442]
 [1311 6278]]
Лучшие параметры для RandomForest: {'

### Шаг 3. Сравнение результатов

1. Сравнить и найти наиболее точные (по F-мере) модель и способ векторизации текстов.

2. Обучить полученную модель (с заданными оптимальными гиперпараметрами) на всех данных (80% обучающая выборка, 20% тестовая выборка).

3. На тестовой выборке постоить матрицу ошибок (confusion matrix) и оценить качество классификации как accuracy, precision, recall и F-мера.

In [36]:
# вставьте код здесь
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# ----------- Сравнение по F-мере -----------
# Сравниваем F-мера для моделей с TF-IDF и CountVectorizer

# Вычисляем F-меру для каждой модели
f1_svc_tfidf = f1_score(y_test, y_pred_svc, average='macro')
f1_rf_tfidf = f1_score(y_test, y_pred_rf_tfidf, average='macro')
f1_rf_count = f1_score(y_test, y_pred_rf_count, average='macro')

# Вывод F-меры для каждой модели
print(f"F-мера для SVC с TF-IDF: {f1_svc_tfidf}")
print(f"F-мера для RandomForest с TF-IDF: {f1_rf_tfidf}")
print(f"F-мера для RandomForest с CountVectorizer: {f1_rf_count}")

# Выбираем модель с наибольшей F-мерой
best_model_name = ""
best_f1_score = max(f1_svc_tfidf, f1_rf_tfidf, f1_rf_count)

if best_f1_score == f1_svc_tfidf:
    best_model_name = "SVC с TF-IDF"
elif best_f1_score == f1_rf_tfidf:
    best_model_name = "RandomForest с TF-IDF"
else:
    best_model_name = "RandomForest с CountVectorizer"

print(f"Наиболее точная модель по F-мере: {best_model_name} с F-мерой {best_f1_score}")


F-мера для SVC с TF-IDF: 0.8458618618763898
F-мера для RandomForest с TF-IDF: 0.8163887493963605
F-мера для RandomForest с CountVectorizer: 0.8141169813392791
Наиболее точная модель по F-мере: SVC с TF-IDF с F-мерой 0.8458618618763898


In [40]:
# Разделение данных на 80% обучающая выборка и 20% тестовая выборка
X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(X, y, test_size=0.2, random_state=42)


In [44]:
if best_model_name == "SVC с TF-IDF":
    # Векторизация TF-IDF
    tfidf_vectorizer = TfidfVectorizer(max_features=500)
    X_train_vec = tfidf_vectorizer.fit_transform(X_train_final)
    X_test_vec = tfidf_vectorizer.transform(X_test_final)

    # Лучшая модель SVC
    svc_model = SVC(C=grid_search_svc.best_params_['C'],
                    kernel=grid_search_svc.best_params_['kernel'],
                    gamma=grid_search_svc.best_params_['gamma'],
                    random_state=42)
    best_model = svc_model

elif best_model_name == "RandomForest с TF-IDF":
    # Векторизация TF-IDF
    tfidf_vectorizer = TfidfVectorizer(max_features=500)
    X_train_vec = tfidf_vectorizer.fit_transform(X_train_final)
    X_test_vec = tfidf_vectorizer.transform(X_test_final)

    # Лучшая модель RandomForest
    rf_model = RandomForestClassifier(n_estimators=grid_search_rf_tfidf.best_params_['n_estimators'],
                                      max_depth=grid_search_rf_tfidf.best_params_['max_depth'],
                                      min_samples_split=grid_search_rf_tfidf.best_params_['min_samples_split'],
                                      min_samples_leaf=grid_search_rf_tfidf.best_params_['min_samples_leaf'],
                                      bootstrap=grid_search_rf_tfidf.best_params_['bootstrap'],
                                      random_state=42)
    best_model = rf_model

else:
    # Векторизация CountVectorizer
    count_vectorizer = CountVectorizer(max_features=500)
    X_train_vec = count_vectorizer.fit_transform(X_train_final)
    X_test_vec = count_vectorizer.transform(X_test_final)

    # Лучшая модель RandomForest
    rf_model = RandomForestClassifier(n_estimators=grid_search_rf_count.best_params_['n_estimators'],
                                      max_depth=grid_search_rf_count.best_params_['max_depth'],
                                      min_samples_split=grid_search_rf_count.best_params_['min_samples_split'],
                                      min_samples_leaf=grid_search_rf_count.best_params_['min_samples_leaf'],
                                      bootstrap=grid_search_rf_count.best_params_['bootstrap'],
                                      random_state=42)
    best_model = rf_model

# Обучаем выбранную лучшую модель
best_model.fit(X_train_vec, y_train_final)

# Прогнозируем на тестовой выборке
y_pred_final = best_model.predict(X_test_vec)



In [46]:
# Прогнозируем на тестовой выборке
y_pred_final = best_model.predict(X_test_vec)


print("\nОценка качества модели на 20% тестовой выборке:")
print("Accuracy:", accuracy_score(y_test_final, y_pred_final))
print("Precision:", precision_score(y_test_final, y_pred_final, average='macro'))
print("Recall:", recall_score(y_test_final, y_pred_final, average='macro'))
print("F-мера:", f1_score(y_test_final, y_pred_final, average='macro'))

print("\nМатрица ошибок:")
print(confusion_matrix(y_test_final, y_pred_final))



Оценка качества модели на 20% тестовой выборке:
Accuracy: 0.8465
Precision: 0.8467158270546882
Recall: 0.8463993949391881
F-мера: 0.8464416094219827

Матрица ошибок:
[[4135  826]
 [ 709 4330]]


In [4]:
from collections import Counter

# Исходные тексты
texts = [
    ['<s>', 'я', 'преподаватель', '</s>'],
    ['<s>', 'кот', 'это', 'не', 'я', '</s>']
]

# Расчет частот для униграмм и биграмм
def get_ngrams(texts, n):
    ngrams = Counter()
    for text in texts:
        ngrams.update(zip(*[text[i:] for i in range(n)]))
    return ngrams

# Число уникальных токенов
vocab = set(word for text in texts for word in text)
V = len(vocab)

# Вычисление частот униграмм и биграмм
unigrams = get_ngrams(texts, 1)
bigrams = get_ngrams(texts, 2)

# Метод для вычисления вероятности биграм
def bigram_prob(bigram, unigram_count, bigram_count, V):
    first_word, second_word = bigram
    return bigram_count.get((first_word, second_word), 0) / unigram_count.get(first_word, 1)

# Линейная интерполяция
def linear_interpolation(prob1, prob2, prob3, lambda1, lambda2, lambda3):
    return lambda1 * prob1 + lambda2 * prob2 + lambda3 * prob3

# Вычисление вероятности P(преподаватель | не я)
# 1. Без учета интерполяции (максимальное правдоподобие)
prob_bi_1 = bigram_prob(('не', 'я'), unigrams, bigrams, V)
prob_bi_2 = bigram_prob(('я', 'преподаватель'), unigrams, bigrams, V)

# 2. Интерполяция
lambda1, lambda2, lambda3 = 0.3, 0.4, 0.3

# Линейная интерполяция (например, для более простого случая без третьего компонента)
final_prob = linear_interpolation(prob_bi_1, prob_bi_2, 0, lambda1, lambda2, lambda3)

# Вывод вероятности
print(f"Вероятность P(преподаватель | не я): {final_prob:.2f}")

Вероятность P(преподаватель | не я): 0.70


In [8]:
import math
from collections import Counter

# Исходные тексты
texts = [
    ['я', 'преподаватель'],
    ['кот', 'это', 'не', 'я'],
    ['это', 'сделал', 'не', 'я'],
    ['я', 'не', 'люблю', 'петь']
]

# Фраза для оценки перплексии
test_phrase = ['это', 'не', 'я', 'преподаватель']

# Функция для получения униграмм
def get_unigrams(texts):
    unigrams = Counter()
    total_words = 0
    for text in texts:
        unigrams.update(text)
        total_words += len(text)
    return unigrams, total_words

# Получаем униграммы и общее количество слов
unigrams, total_words = get_unigrams(texts)

# Вычисляем вероятность униграммы
def unigram_prob(word, unigram_count, total_words):
    return unigram_count.get(word, 0) / total_words

# Вычисление логарифма вероятности для фразы
log_probs = []
for word in test_phrase:
    prob = unigram_prob(word, unigrams, total_words)
    log_probs.append(math.log(prob) if prob > 0 else float('-inf'))

# Средняя логарифмическая вероятность
avg_log_prob = sum(log_probs) / len(test_phrase)

# Перплексия
perplexity = math.exp(-avg_log_prob)

# Выводим результат с точностью до 2 знаков после запятой
print(f"Перплексия фразы 'это не я преподаватель': {perplexity:.2f}")

Перплексия фразы 'это не я преподаватель': 6.33
